<a href="https://colab.research.google.com/github/kuostar0620-jpg/114-1KUO-REPO-/blob/main/%E4%BD%9C%E6%A5%AD%E4%BA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install -q gradio pandas

import pandas as pd
import re
import datetime
import gradio as gr
import urllib.parse

# ==============================
# 1. 讀取 Google Sheet
# ==============================

SHEET_URL = "https://docs.google.com/spreadsheets/d/1Ncmeu-R92aV4i-spzUcYrzsMwBSBZGsuPiqgJBwYHS0/export?format=csv"

def fetch_sheet():
    df = pd.read_csv(SHEET_URL)
    if "店家名稱" in df.columns:
        df = df[df["店家名稱"].notna()]
    return df

df_lunch = fetch_sheet()

# ==============================
# 2. 處理價位
# ==============================

def extract_price(val):
    if pd.isna(val):
        return None
    numbers = list(map(int, re.findall(r"\d+", str(val))))
    return sum(numbers)/len(numbers) if numbers else None

df_lunch["平均價位"] = df_lunch["均價"].apply(extract_price)

# ==============================
# 3. 處理營業時間
# ==============================

weekday_dict = {"週一":0,"週二":1,"週三":2,"週四":3,"週五":4,"週六":5,"週日":6}
weekday_alias = {
    0:["週一","星期一","禮拜一"],1:["週二","星期二","禮拜二"],2:["週三","星期三","禮拜三"],
    3:["週四","星期四","禮拜四"],4:["週五","星期五","禮拜五"],5:["週六","星期六","禮拜六"],
    6:["週日","星期日","禮拜日"]
}

def clean_hours(s):
    s = str(s).replace("：", ":")
    for ch in ["~","～","至","–","—"]:
        s = s.replace(ch,"-")
    return s

def closed_on_day(s, weekday_idx):
    s = str(s)
    if "不定休" in s or "無固定公休" in s:
        return False
    if "公休" not in s:
        return False
    return any(kw in s for kw in weekday_alias.get(weekday_idx, []))

def parse_hours(s):
    s = clean_hours(s)
    if pd.isna(s) or s=="":
        return []
    if any(x in s for x in ["24小時","全天","全日"]):
        return [(datetime.time(0,0), datetime.time(23,59))]
    pattern = r"(\d{1,2}):(\d{2})\s*-\s*(\d{1,2}):(\d{2})"
    times = []
    for h1,m1,h2,m2 in re.findall(pattern,s):
        h1,m1,h2,m2 = int(h1),int(m1),int(h2),int(m2)
        if h1==24: h1=23; m1=59
        if h2==24: h2=23; m2=59
        if 0<=h1<=23 and 0<=m1<=59 and 0<=h2<=23 and 0<=m2<=59:
            times.append((datetime.time(h1,m1), datetime.time(h2,m2)))
    return times

df_lunch["時間區間"] = df_lunch.get("營業時間", pd.Series([""]*len(df_lunch))).apply(parse_hours)

def within_range(t, rng):
    start,end = rng
    if start<=end:
        return start<=t<=end
    return t>=start or t<=end

def is_open_now(biz_hours, ranges, weekday_idx, query_time):
    if closed_on_day(biz_hours, weekday_idx):
        return False
    if not ranges:
        return True
    return any(within_range(query_time,r) for r in ranges)

# ==============================
# 4. AI 簡短摘要
# ==============================

def summarize_top3(df3):
    texts = []
    for _, r in df3.iterrows():
        name = r.get("店家名稱","").strip()
        price = r.get("平均價位")
        cate = r.get("販售內容","").strip()
        feat = r.get("特色","").strip()
        pros, cons = [], []
        if price is not None:
            if price<=100: pros.append("超平價")
            elif price<=200: pros.append("價格親民")
            else: cons.append("稍偏貴")
        if cate: pros.append(cate[:8])
        if feat: pros.append(feat[:8])
        desc = f"{name}："
        if pros: desc+="優點：" + "、".join(pros)
        if cons: desc+="；缺點：" + "、".join(cons)
        texts.append(desc)
    out = " / ".join(texts)
    return out[:100]+"..." if len(out)>100 else out

# ==============================
# 5. Google Map iframe
# ==============================

def maps_embed(df3):
    blocks = []
    for _, r in df3.iterrows():
        name = r.get("店家名稱","").strip()
        addr = r.get("地點","").strip()
        if not addr: continue
        q = urllib.parse.quote_plus(addr)
        url = f"https://www.google.com/maps?q={q}&output=embed"
        html = (
            f"<div style='margin-bottom:20px;'>"
            f"<div><b>{name}</b> - {addr}</div>"
            f"<iframe src='{url}' width='100%' height='260' style='border:0;' loading='lazy'></iframe>"
            f"</div>"
        )
        blocks.append(html)
    if not blocks: return "<p>沒有可顯示的地圖。</p>"
    return "".join(blocks)

# ==============================
# 6. 主決策函式（只依預算與時間）
# ==============================

def decide_lunch(max_price, time_input, weekday_sel):
    if not time_input:
        return "請輸入時間，例如 12:30", "", pd.DataFrame(), "<p>尚未產生地圖。</p>"
    try:
        h,m = map(int,time_input.split(":"))
        qtime = datetime.time(h,m)
    except:
        return "時間格式錯誤", "", pd.DataFrame(), "<p>時間格式錯誤，無法產生地圖。</p>"
    weekday_idx = weekday_dict.get(weekday_sel)
    if weekday_idx is None:
        return "請選擇星期幾", "", pd.DataFrame(), "<p>尚未產生地圖。</p>"
    df = df_lunch.copy()
    df = df[df["平均價位"].notna() & (df["平均價位"]<=max_price)]
    df = df[df.apply(lambda r:is_open_now(r.get("營業時間",""), r.get("時間區間",[]), weekday_idx, qtime), axis=1)]
    if df.empty:
        return "找不到符合條件的店家", "", pd.DataFrame(), "<p>沒有符合條件的店家，因此無地圖可顯示。</p>"
    df = df.head(3)
    display_cols = ["店家名稱","販售內容","地點","特色","均價"]
    result = df[display_cols].reset_index(drop=True)
    return f"幫你挑出 {len(result)} 家候選午餐（依預算篩選）", summarize_top3(df), result, maps_embed(df)

# ==============================
# 7. Gradio 介面
# ==============================

weekdays = list(weekday_dict.keys())

with gr.Blocks() as app:
    gr.Markdown("## 師大午餐決策器 🍱\n輸入預算與用餐時間，幫你挑出 3 家目前有開的餐廳，並附上 AI 簡短建議與地圖預覽。")

    with gr.Row():
        budget = gr.Slider(50,1000,step=50,value=200,label="預算上限 (NT$)")

    with gr.Row():
        time_input = gr.Textbox(value="12:30", label="用餐時間 (HH:MM)")
        weekday_input = gr.Dropdown(weekdays,value="週一",label="星期幾")

    btn = gr.Button("幫我決定吃甚麼~")
    msg_out = gr.Markdown(label="篩選結果")
    ai_out = gr.Markdown(label="AI 建議")
    table_out = gr.Dataframe(label="推薦 3 家", interactive=False)
    map_out = gr.HTML()

    btn.click(fn=decide_lunch, inputs=[budget,time_input,weekday_input],
              outputs=[msg_out, ai_out, table_out, map_out])

app.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>